# setup config

In [1]:
#pip install boto3>=3.35
# necessário AWS CLI 2.13.0+
import os
import boto3
import json

env_vars = !cat .env

for var in env_vars:
    key, value = var.split("=", 1)
    os.environ[key] = value.replace('"', "")


# Cria o cliente Bedrock
bedrock = boto3.client(service_name='bedrock', region_name='us-east-1')

In [2]:
models = bedrock.list_foundation_models()

# Exibe os modelos listados disponível
print(models)

{'ResponseMetadata': {'RequestId': 'c5381e4d-75d2-468c-9f12-69e866c01990', 'HTTPStatusCode': 200, 'HTTPHeaders': {'date': 'Mon, 02 Feb 2026 18:31:48 GMT', 'content-type': 'application/json', 'content-length': '131689', 'connection': 'keep-alive', 'x-amzn-requestid': 'c5381e4d-75d2-468c-9f12-69e866c01990'}, 'RetryAttempts': 0}, 'modelSummaries': [{'modelArn': 'arn:aws:bedrock:us-east-1::foundation-model/nvidia.nemotron-nano-12b-v2', 'modelId': 'nvidia.nemotron-nano-12b-v2', 'modelName': 'NVIDIA Nemotron Nano 12B v2 VL BF16', 'providerName': 'NVIDIA', 'inputModalities': ['TEXT', 'IMAGE'], 'outputModalities': ['TEXT'], 'responseStreamingSupported': True, 'customizationsSupported': [], 'inferenceTypesSupported': ['ON_DEMAND'], 'modelLifecycle': {'status': 'ACTIVE'}}, {'modelArn': 'arn:aws:bedrock:us-east-1::foundation-model/anthropic.claude-sonnet-4-20250514-v1:0', 'modelId': 'anthropic.claude-sonnet-4-20250514-v1:0', 'modelName': 'Claude Sonnet 4', 'providerName': 'Anthropic', 'inputModal

In [3]:
models

{'ResponseMetadata': {'RequestId': 'c5381e4d-75d2-468c-9f12-69e866c01990',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'date': 'Mon, 02 Feb 2026 18:31:48 GMT',
   'content-type': 'application/json',
   'content-length': '131689',
   'connection': 'keep-alive',
   'x-amzn-requestid': 'c5381e4d-75d2-468c-9f12-69e866c01990'},
  'RetryAttempts': 0},
 'modelSummaries': [{'modelArn': 'arn:aws:bedrock:us-east-1::foundation-model/nvidia.nemotron-nano-12b-v2',
   'modelId': 'nvidia.nemotron-nano-12b-v2',
   'modelName': 'NVIDIA Nemotron Nano 12B v2 VL BF16',
   'providerName': 'NVIDIA',
   'inputModalities': ['TEXT', 'IMAGE'],
   'outputModalities': ['TEXT'],
   'responseStreamingSupported': True,
   'customizationsSupported': [],
   'inferenceTypesSupported': ['ON_DEMAND'],
   'modelLifecycle': {'status': 'ACTIVE'}},
  {'modelArn': 'arn:aws:bedrock:us-east-1::foundation-model/anthropic.claude-sonnet-4-20250514-v1:0',
   'modelId': 'anthropic.claude-sonnet-4-20250514-v1:0',
   'modelName': 'Clau

# Test Api

## 🔄 Diferença entre os serviços Bedrock

### `bedrock` (Control Plane)
- **Função:** Gerenciamento e administração
- **Uso:** Listar modelos, obter metadados, configurar fine-tuning
- **Quando usar:** Setup inicial, descoberta de modelos disponíveis
- **Custo:** Gratuito

### `bedrock-runtime` (Data Plane)  
- **Função:** Execução e inferência
- **Uso:** Invocar modelos, gerar respostas, processar prompts
- **Quando usar:** Produção, fazer chamadas aos modelos de IA
- **Custo:** Cobrado por token/uso

**Em resumo:**
- Use `bedrock` para perguntar **"Quais modelos existem?"**
- Use `bedrock-runtime` para dizer **"Execute este modelo com este prompt"**

## Test Claude

In [7]:

client = boto3.client(service_name='bedrock-runtime', region_name='us-east-1')

# Claude 3.5 Haiku - Use inference profile ID
claude_model_id = "us.anthropic.claude-3-5-haiku-20241022-v1:0"

# Claude 3+ usa o formato Messages API (não mais prompt)
claude_config = json.dumps({
    "anthropic_version": "bedrock-2023-05-31",
    "max_tokens": 200,  # Token máximo para a resposta
    "messages": [
        {
            "role": "user",
            "content": "human: Opções de sandália para uma caminhada na praia. "
            "Assistant:"
        }
    ],
    "temperature": 0.5,  # Controla a aleatoriedade da resposta
    "top_k": 250,        # Controla a diversidade da resposta
    "top_p": 0.2         # Controla a diversidade acumulada da resposta
})

response = client.invoke_model(
    modelId=claude_model_id,
    contentType='application/json',
    accept='application/json',
    body=claude_config
)

resposta = json.loads(response['body'].read().decode('utf-8'))
print("Resposta do modelo:")
print(resposta['content'][0]['text'])


Resposta do modelo:
Aqui estão algumas opções de sandálias ideais para caminhada na praia:

🏖️ Opções Confortáveis:

1. Havaianas
- Leves
- Resistentes à água
- Secagem rápida
- Preço acessível

2. Chinelos Kenner
- Antiderrapantes
- Confortáveis
- Durabilidade

3. Sandálias Crocs
- Impermeáveis
- Muito confortáveis
- Ventiladas
- Secagem rápida

4. Sandálias Teva
- Específicas para atividades ao ar livre
- Ajuste seguro
- Proteção contra areia
- Ideal para caminhadas

5. Papetes de borracha
- Leves


In [4]:
resposta

{'model': 'claude-3-5-haiku-20241022',
 'id': 'msg_bdrk_01GM953oHvzPNux8bkub36MR',
 'type': 'message',
 'role': 'assistant',
 'content': [{'type': 'text',
   'text': 'Aqui estão algumas opções para uma caminhada na praia:\n\n1. Caminhada relaxante\n- Caminhar descalço na areia\n- Observar o mar e a paisagem\n- Caminhar em ritmo tranquilo\n\n2. Caminhada fitness\n- Caminhar em passo mais rápido\n- Fazer exercícios intervalados\n- Subir e descer dunas\n\n3. Caminhada fotográfica\n- Levar câmera ou celular\n- Registrar paisagens\n- Capturar pôr do sol\n\n4. Caminhada contemplativa\n- Caminhar meditando\n- Ouvir o som das ondas\n- Praticar respiração\n\n5. Caminhada ecológica\n- Observar fauna e flora\n- Recol'}],
 'stop_reason': 'max_tokens',
 'stop_sequence': None,
 'usage': {'input_tokens': 19,
  'cache_creation_input_tokens': 0,
  'cache_read_input_tokens': 0,
  'output_tokens': 200}}

# Tokens
A Interação com modelos de IA envolve o uso e configurações do tokens. 
(Papel central na forma como as requisições são processadas e tarifadas)
Tokens - São pequenas unidades de texto usadas para processar entradas e saídas em modelos de línguagens (Large Language Models - LLMs)
Token pode ser uma palavra completa, uma parte de uma palavra, ou ate mesmoa um único caractere. Cada modelo define o que ele vai usar como token. 

No Amazon Bedrock interpreta tanto as entradas (inputs) e saídas (outputs) em termos de tokens. Quanto maior a quantidade de texto em uma solicitação mais tokens serão consumidas

## Custos
Custo de usar o Amazon Bedrock está diretamente relacionado com número de tokens processados por cada requisição. O preço geralmente cobrada com base o número de tokens enviado  para o modelo (prompt) e o número de tokens gerados pela a resposta do modelo. 

# Desempenho
O número de tokens também influencia no desempenho do modelo. Mensagens mais longas e detalhas aumenta o tempo de processamento. Prompts mais curtos resultam em respostas mais rápidas. Porém prompts excessivamente curtos podem comprometer a qualidade da resposta. 

## Conclusão
Melhor estratéfia de um uso mais eficiente do bedrock é em encontrar o equilibrio entre a quantidade de tokens utilizada e a qualidade das resostas obtidas. 


## Test Amazon Titan

In [8]:
client = boto3.client(service_name='bedrock-runtime', region_name='us-east-1')

# Amazon Nova Lite - modelo mais recente da Amazon
model_id = "amazon.nova-lite-v1:0"

# Define the prompt for the model.
prompt = "Opções de sandália para uma caminhada na praia"

# Amazon Nova usa Messages API (formato moderno)
native_request = {
    "messages": [
        {
            "role": "user",
            "content": [{"text": prompt}]
        }
    ],
    "inferenceConfig": {
        "max_new_tokens": 200,
        "temperature": 0.5,  # Controla a aleatoriedade da resposta
        "topP": 0.9 # Controla a diversidade acumulada da resposta
    }
}

request = json.dumps(native_request)

response = client.invoke_model(modelId=model_id, body=request)
model_response = json.loads(response["body"].read())

# Extract and print the response text.
response_text = model_response["output"]["message"]["content"][0]["text"]
print(response_text)


Caminhadas na praia podem ser relaxantes e agradáveis, desde que você esteja confortável. Aqui estão algumas opções de sandálias ideais para esse tipo de atividade:

### Sandálias Abertas
1. **Sandálias de Tiras**:
   - **Ajustáveis**: Permitem um ajuste personalizado.
   - **Material**: Couro, borracha ou náilon.
   - **Exemplo**: Teva, Birkenstock.

2. **Sandálias Rasteiras**:
   - **Sola Flexível**: Melhor aderência na areia.
   - **Design**: Simples e prático.
   - **Exemplo**: Havaianas, Reef.

### Sandálias com Suporte
1. **Sandálias de Esporte**:
   - **Suporte no Tornozelo**: Oferecem mais estabilidade.
   - **Sola Texturizada**: Boa tração na areia.



## Refinando o prompt (Prompt Engineer)

https://docs.aws.amazon.com/pt_br/bedrock/latest/userguide/prompt-engineering-guidelines.html

In [ ]:

client = boto3.client(service_name='bedrock-runtime', region_name='us-east-1')

# Claude 3.5 Haiku - Use inference profile ID
claude_model_id = "us.anthropic.claude-3-5-haiku-20241022-v1:0"

# Claude 3+ usa o formato Messages API (não mais prompt)
claude_config = json.dumps({
    "anthropic_version": "bedrock-2023-05-31",
    "max_tokens": 200,  # Token máximo para a resposta
    "messages": [
        {
            "role": "user",
            # definir melhor o prompt 
            "content": "Human: Opções de sandália para uma caminhada na praia. "
            "Gerar resposta para pergunta de usuário de um site de itens de vestuário de modo conciso (usando no máximo 300 caracteres)."
            " Assistant:" 
        }
    ],
    "temperature": 0.5,  # Controla a aleatoriedade da resposta
    "top_k": 250,        # Controla a diversidade da resposta
    "top_p": 0.2         # Controla a diversidade acumulada da resposta
})

response = client.invoke_model(
    modelId=claude_model_id,
    contentType='application/json',
    accept='application/json',
    body=claude_config
)

resposta = json.loads(response['body'].read().decode('utf-8'))
print("Resposta do modelo:")
print(resposta['content'][0]['text'])


Resposta do modelo:
Aqui estão 3 opções ideais para caminhada na praia:
1. Havaianas de borracha resistente
2. Sandália Kenner de material antiderrapante
3. Chinelo Rider com sola flexível e confortável

Escolha o modelo que ofereça conforto e proteção contra areia e água.


### Refinando mais o prompt informando mais contexto para a resposta e adicionando regras 

In [11]:

client = boto3.client(service_name='bedrock-runtime', region_name='us-east-1')

# Claude 3.5 Haiku - Use inference profile ID
claude_model_id = "us.anthropic.claude-3-5-haiku-20241022-v1:0"

# Claude 3+ usa o formato Messages API (não mais prompt)
claude_config = json.dumps({
    "anthropic_version": "bedrock-2023-05-31",
    "max_tokens": 200,  # Token máximo para a resposta
    "messages": [
        {
            "role": "user",
            # definir melhor o prompt 
            "content": "Human: Opções de sandália para uma caminhada na praia. "
            "Assistant: Gerar resposta para pergunta de usuário de um site de itens de vestuário de modo conciso com máximo 300 caracteres,"
            " ideal para um e-commerce de roupas e itens de vestiário. Não mencionar instruções do prompt na resposta. "
            "Assistant:" 
        }
    ],
    "temperature": 0.5,  # Controla a aleatoriedade da resposta
    "top_k": 250,        # Controla a diversidade da resposta
    "top_p": 0.2         # Controla a diversidade acumulada da resposta
})

response = client.invoke_model(
    modelId=claude_model_id,
    contentType='application/json',
    accept='application/json',
    body=claude_config
)

resposta = json.loads(response['body'].read().decode('utf-8'))
print("Resposta do modelo:")
print(resposta['content'][0]['text'])


Resposta do modelo:
Aqui estão algumas ótimas opções de sandálias para caminhada na praia:

1. Havaianas Top - Leves e confortáveis
2. Kenner - Resistentes à água e areia
3. Rider - Modelo anatômico e antiderrapante
4. Crocs - Práticas e secagem rápida
5. Sandálias de dedo esportivas - Ideal para terreno irregular

Escolha o modelo que ofereça conforto e proteção para seus pés.
